In [1]:
import json
from itertools import groupby

In [2]:
with open("./cards.js", "r") as datafile:
    data=json.load(datafile)["fow"]["clusters"][::-1]

In [11]:
#Testing cell
data[0]["sets"][0]["cards"][0]

{'id': 'EDL-001',
 'name': 'Dispelling Stone',
 'type': 'Chant',
 'race': ['Treasure'],
 'cost': '{W}',
 'ATK': '',
 'DEF': '',
 'abilities': ['[Quickcast]',
  "Destroy target addition. Draw a card. Shuffle this card into its owner's deck as it resolves."],
 'divinity': '',
 'flavor': '"Oh, it ... It\'s wonderful! ... I think." - Selesta, Treasure Hunter',
 'rarity': 'N'}

In [12]:
#Temporary, dump code->name dictionary until this can be done on the go
names={}
for cluster in data:
    for setd in cluster["sets"]:
        for card in setd["cards"]:
            names[card["id"]]=card["name"]

with open('names.js', 'w') as json_file:
    json_file.write("\nnames=")
    json.dump(names, json_file, indent=2, sort_keys=True)

In [5]:
#Sort all cards within a set
def sortset(setdata):
    rarities=[[i["id"], i["rarity"]] for i in setdata["cards"]]
    rarities.sort(key=lambda x:x[1])
    grouped=groupby(rarities, key=lambda x:x[1])
    return [{'rarity':k, 'cards':[x[0] for x in v]} for k, v in grouped]

In [6]:
#Group cards by rarity
for n,i in enumerate(data):
    for m,j in enumerate(data[n]["sets"]):
        data[n]["sets"][m]["cards"]=sortset(data[n]["sets"][m])

In [7]:
#Merge rarity and card fields
for n,i in enumerate(data):
    for m,j in enumerate(i["sets"]):
        for mm,k in enumerate(j["cards"]):
            data[n]["sets"][m]["cards"][mm][k["rarity"]]=k["cards"]
            del data[n]["sets"][m]["cards"][mm]["rarity"]
            del data[n]["sets"][m]["cards"][mm]["cards"]
        data[n]["sets"][m]["cards"]={list(group)[0]:group[list(group)[0]] for group in j["cards"]}

In [13]:
#Remove starter decks, vingolf sets and promos
#May need to run this one twice for some reason
remcodes=["SDAO1", "SDAO2", "AOPR", "GITS2045SD", "SDV1", "SDV2", "SDV3", "SDV4", "SDV5", "SDR1", "SDR2", "SDR3", "SDR4", "SDR5", "SDR6", "SDL1", "SDL2", "SDL3", "SDL4", "SDL5", "VIN001", "VIN002", "VIN003", "VS01", "S"]
for n,i in enumerate(data):
    for m,j in enumerate(i["sets"]):
        if j["code"] in remcodes: del data[n]["sets"][m]

In [14]:
#(Optional, make sure all the right sets are left)
for n,i in enumerate(data):
    for m,j in enumerate(i["sets"]):
        print(j["code"], j["name"])

EDL The Epic of the Dragon Lord
AO1 Alice Origin
AO2 Alice Origin II
GITS2045 GHOST IN THE SHELL SAC_2045
AO3 Alice Origin III
PofA Prologue of Attoractia
NDR New Dawn Rises
SNV The Strangers of New Valhalla
AOA Awakening of the Ancients
DBV The Decisive Battle of Valhalla
ACN Ancient Nights
ADK Advent of the Demon King
TSW The Time-Spinning Witch
WOM Winds of the Ominous Moon
CFC Curse of the Frozen Casket
LEL Legacy Lost
RDE Return of the Dragon Emperor
ENW Echoes of the New World
SKL The Seven Kings of the Land
TTW The Twilight Wanderer
TMS The Moonlit Saviour
BFA Battle for Attoractia
CMF The Crimson Moon Fairy Tale
TAT The Castle of Heaven and the Two Towers
MPR The Moon Priestess Returns
MOA The Millennia of Ages
1 The Dawn of Valhalla
2 The War of Valhalla
3 The Shaft of Light of Valhalla


In [24]:
with open('out.js', 'w') as json_file:
    json_file.write("data=")
    json.dump(data, json_file, indent=2, sort_keys=True)

In [21]:
#Add list of ruler cards for easier exclusions
rulers=[]
for cluster in data:
    for sets in cluster["sets"]:
        for rarity in sets["cards"].keys():
            for card in sets["cards"][rarity]:
                if card[-1]=="J": rulers.append(card[:-1])

In [25]:
with open('out.js', 'a') as json_file:
    json_file.write("\nrulers=")
    json.dump(rulers, json_file, indent=2, sort_keys=True)